In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from syft.workers.node_client import NodeClient
import syft as sy  # import the Pysyft library

In [2]:
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False
server = hook.local_worker

In [3]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

In [4]:
alice = NodeClient(hook, "ws://localhost:6666" , id="alice")
bob = NodeClient(hook, "ws://localhost:6667" , id="bob")
charlie = NodeClient(hook, "ws://localhost:6668" , id="charlie")
grid = sy.PrivateGridNetwork(alice, bob, charlie)

In [5]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [6]:
net = Net()
net_2 = Net()

net.build(torch.tensor([1., 2.]))
net_2.build(torch.tensor([1., 2.]))

PlaceHolder[Id:19838256510]>tensor([-0.9544, -0.4862], grad_fn=<LogSoftmaxBackward>)

In [7]:
print(net.parameters())

[Parameter containing:
tensor([[ 0.2663,  0.0383],
        [ 0.2315, -0.6249],
        [ 0.7035,  0.3375]], requires_grad=True), Parameter containing:
tensor([ 0.2425,  0.1366, -0.0398], requires_grad=True), Parameter containing:
tensor([[ 0.2589,  0.4239,  0.4886],
        [-0.5572,  0.5263, -0.5643]], requires_grad=True), Parameter containing:
tensor([0.2787, 0.4987], requires_grad=True)]


In [8]:
print(net_2.parameters())

[Parameter containing:
tensor([[ 0.6512, -0.3418],
        [ 0.0445,  0.2618],
        [ 0.2344,  0.4595]], requires_grad=True), Parameter containing:
tensor([ 0.3079,  0.4547, -0.4008], requires_grad=True), Parameter containing:
tensor([[-0.5297,  0.1098, -0.4748],
        [ 0.3195, -0.4971,  0.2007]], requires_grad=True), Parameter containing:
tensor([0.0203, 0.3668], requires_grad=True)]


In [9]:
len(net.parameters())

4

In [10]:
enc_params = []
params = list(net.parameters())
for param_index in range(len(params)):
    copy_para = params[param_index].copy()
    fix_para = copy_para.fix_precision(precision_fractional=5)
    enc_para = fix_para.share(alice, bob, charlie)
    enc_params.append(enc_para)

In [11]:
enc_params_2 = []
params_2 = list(net_2.parameters())
for param_index in range(len(params_2)):
    copy_para_2 = params_2[param_index].copy()
    fix_para_2 = copy_para_2.fix_precision(precision_fractional=5)
    enc_para_2 = fix_para_2.share(alice, bob, charlie)
    enc_params_2.append(enc_para_2)

In [12]:
len(enc_params)

4

In [13]:
len(enc_params_2)

4

In [14]:
enc_params_3 = []
for param_index in range(len(enc_params)):
    enc_params_3.append(enc_params[param_index] + enc_params_2[param_index])

In [15]:
len(enc_params_3)

4

In [16]:
params_3 = []
for param_index in range(len(enc_params_3)):
    decrypt_para_sum = enc_params_3[param_index].get()
    float_para_sum = decrypt_para_sum.float_precision()
    params_3.append(float_para_sum)